<b> TO BE DONE </b>

# MODEL-BASED META-LEARNING

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
#%cd drive/MyDrive/'Colab Notebooks'
#%cd meta-learning-course-notebooks/1_MAML/
#!ls

In [ ]:
#!pip install import_ipynb --quiet

In [ ]:
#!pip install learn2learn --quiet

In [1]:
import import_ipynb
import utils
import models
utils.hide_toggle('Imports 1')

importing Jupyter notebook from utils.ipynb
importing Jupyter notebook from models.ipynb


In [2]:
from IPython import display
import torch
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
from l2lutils import KShotLoader
from IPython import display
import torch.nn as nn
utils.hide_toggle('Imports 2')

importing Jupyter notebook from l2lutils.ipynb


# Data Generation/Loading

In [3]:
#Generate data - euclidean
meta_train_ds, meta_test_ds, full_loader = utils.euclideanDataset(n_samples=10000,n_features=20,n_classes=10,batch_size=32)

In [3]:
#Generate data - sinusoidal mix
meta_train_ds, meta_test_ds, full_loader = utils.sinDataset(n_samples=10000,length=20,n_classes=10,batch_size=16)

In [4]:
meta_train_ds, meta_test_ds, full_loader = utils.mnist_data(frac=0.2)

In [10]:
# Define an MLP network. Note that input dimension has to be data dimension. For classification
# final dimension has to be number of classes; for regression one.
#torch.manual_seed(10)
net = models.MLP(dims=[20,64,10])

In [11]:
# Train the network; note that network is trained in place so repeated calls further train it.
net,losses,accs=models.Train(net,full_loader,lr=1e-2,epochs=50,verbose=True)

Epoch   49 Loss: 2.79184e-02 Accuracy: 0.99441


In [12]:
#Training accuracy.
models.accuracy(net,meta_train_ds.samples,meta_train_ds.labels,verbose=True)

7461.0 7500


0.9948

In [13]:
# Test accuracy.
models.accuracy(net,meta_test_ds.samples,meta_test_ds.labels)

2437.0 2500


0.9748

# Meta-Learning: Tasks

Generate a k-shot n-way loader using the meta-training dataset

In [78]:
meta_train_kloader=KShotLoader(meta_train_ds,shots=5,ways=2)

Sample a task - each task has a k-shot n-way training set and a similar test set

In [79]:
d_train,d_test=meta_train_kloader.get_task()

Let's try directly learning using the task training set albeit its small size: create a dataset and loader and train it with the earlier network and Train function.

In [16]:
taskds = utils.MyDS(d_train[0],d_train[1])

In [17]:
d_train_loader = torch.utils.data.DataLoader(dataset=taskds,batch_size=1,shuffle=True)

In [20]:
net,losses,accs=models.Train(net,d_train_loader,lr=1e-1,epochs=10,verbose=True)

Epoch   9 Loss: 2.16691e-03 Accuracy: 1.00000


How does it do on the test set of the sampled task?

In [21]:
models.accuracy(net,d_test[0],d_test[1])

12.0 25


0.48

# RNN-based  Meta-learning

In [4]:
# optimisers from torch
import torch.optim as optim
import torch.nn.functional as F

In [5]:
lossfn = torch.nn.NLLLoss()

Get a task dataset.

In [80]:
d_train,d_test=meta_train_kloader.get_task()

In [81]:
torch.eye(5)[d_train[1]]

tensor([[0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.]])

In [82]:
x_tr = d_train[0]
y_tr_sh = torch.cat((torch.zeros(1,5),torch.eye(5)[d_train[1][1:]]))
d_tr = torch.cat((x_tr,y_tr_sh),1).unsqueeze(0)
d_tr.shape

torch.Size([1, 10, 25])

Recurrent meta-learning network using GRU:

In [83]:
class MetaRNN(nn.Module):
    def __init__(self,n_features=1,dim=5,n_layers=2,n_classes=2,lr=1e-3):
        super(MetaRNN,self).__init__()
        self.n_layers = n_layers
        self.dim = dim
        self.n_features = n_features
        self.rnn = nn.GRU(n_features, dim, n_layers,batch_first=True)
        self.linear = nn.Linear(dim,n_classes)
        #self.linear2 = nn.Linear(dim,n_classes)
        self.logsoft = nn.LogSoftmax(dim=-1)
        self.optimizer = optim.Adam(self.parameters(),lr=lr)
        self.H = 0
    def adapt(self,X):
        if self.n_features==1: X=X.unsqueeze(-1)
        h = torch.randn(self.n_layers,X.shape[0],self.dim)
        R = self.rnn(X,h)
        self.H = R[1]
        G = self.logsoft(self.linear(R[0][-1,:,:]))
        return G
    def forward(self,Y):
        if self.n_features==1: Y=Y.unsqueeze(-1)
        h = self.H.repeat(1,Y.shape[0],1)
        H = self.rnn(Y,h)[1]
        #YH = torch.cat((Y,self.H.squeeze().repeat(Y.shape[0],1)),dim=1)
        G = self.logsoft(self.linear(H[-1,:,:]))
        return G
utils.hide_toggle('Class MetaRNN')

In [84]:
net = MetaRNN(n_features=25,dim=32,n_layers=2,n_classes=5)

In [85]:
d_tr.shape

torch.Size([1, 10, 25])

In [87]:
net.adapt(d_tr)

tensor([[-1.4046, -1.9978, -1.7352, -1.1966, -1.9641],
        [-1.4969, -1.9738, -1.5423, -1.2812, -1.9264],
        [-1.4638, -1.8917, -1.4706, -1.4604, -1.8585],
        [-1.4629, -1.8077, -1.4419, -1.6391, -1.7500],
        [-1.5763, -1.7180, -1.4604, -1.6710, -1.6417],
        [-1.5688, -1.6912, -1.5446, -1.5566, -1.6974],
        [-1.6767, -1.7307, -1.5502, -1.4857, -1.6230],
        [-1.6788, -1.7575, -1.5471, -1.4686, -1.6206],
        [-1.7384, -1.7476, -1.5931, -1.4303, -1.5729],
        [-1.7260, -1.7656, -1.5954, -1.4204, -1.5776]],
       grad_fn=<LogSoftmaxBackward>)

In [71]:
x_ts = d_test[0]
y_ts_sh = torch.cat((torch.zeros(1,5),torch.eye(5)[d_test[1][1:]]))
d_ts = torch.cat((x_ts,y_ts_sh),1).unsqueeze(0)
d_tst = d_ts.transpose(0,1)

In [72]:
net(d_tst)

tensor([[-1.3580, -1.5737, -1.9797, -1.6276, -1.6043],
        [-1.4467, -1.5260, -1.8509, -1.7263, -1.5502],
        [-1.3854, -1.5374, -1.8727, -1.6579, -1.6576],
        [-1.3509, -1.5378, -1.9649, -1.6772, -1.6141],
        [-1.3626, -1.5288, -1.9011, -1.6762, -1.6569],
        [-1.4147, -1.5852, -1.8353, -1.6604, -1.5971],
        [-1.4390, -1.4768, -1.8588, -1.6957, -1.6340],
        [-1.4209, -1.5337, -1.8700, -1.6891, -1.5897],
        [-1.3588, -1.5485, -1.9012, -1.6929, -1.6240],
        [-1.4791, -1.4463, -1.8121, -1.7273, -1.6313],
        [-1.3365, -1.4970, -1.9551, -1.7075, -1.6578],
        [-1.3472, -1.5937, -1.9322, -1.6448, -1.6146],
        [-1.3482, -1.5929, -1.9156, -1.6533, -1.6180],
        [-1.3478, -1.6392, -1.9392, -1.6121, -1.5958],
        [-1.3189, -1.5446, -1.9433, -1.6522, -1.6912],
        [-1.3946, -1.5239, -1.9424, -1.6652, -1.6010],
        [-1.4694, -1.5483, -1.8042, -1.6300, -1.6256],
        [-1.3571, -1.5789, -1.9666, -1.6729, -1.5665],
        [-

In [55]:
net(d_tst[0].unsqueeze(0))

tensor([[-1.5979, -1.4704, -1.5418, -1.7355, -1.7286]],
       grad_fn=<LogSoftmaxBackward>)

In [242]:
preds = net(d_test[0])
train_loss = lossfn(preds,d_test[1])
train_loss

tensor(2.4523, grad_fn=<NllLossBackward>)

# Putting it all together: RNN-based Meta-learning
Now let's put all of the above in a loop - the MAML algorithm:

In [97]:
import learn2learn as l2l
import torch.optim as optim
shots,ways = 5,5
net = MetaRNN(n_features=20+ways,n_classes=ways,dim=10,n_layers=2,lr=1e-3)
lossfn = torch.nn.NLLLoss()
meta_train_kloader=KShotLoader(meta_train_ds,shots=shots,ways=ways,num_tasks=1000)

In [101]:
epoch=0
n_epochs=100
task_count=10
fas=1
while epoch<n_epochs:
    test_loss = 0.0
    test_acc = 0.0
    # Sample and train on a task
    for task in range(task_count):
        d_train,d_test=meta_train_kloader.get_task()
        x_tr = d_train[0]
        y_tr_sh = torch.cat((torch.zeros(1,ways),torch.eye(ways)[d_train[1][1:]]))
        d_tr = torch.cat((x_tr,y_tr_sh),1).unsqueeze(0)
        train_preds = net.adapt(d_tr)
        x_ts = d_test[0]
        y_ts_sh = torch.cat((torch.zeros(1,ways),torch.eye(ways)[d_test[1][1:]]))
        d_ts = torch.cat((x_ts,y_ts_sh),1).unsqueeze(0)
        d_tst = d_ts.transpose(0,1)
        test_preds = net(d_tst)
        test_loss += lossfn(test_preds,d_test[1])+lossfn(train_preds,d_train[1])
        #net.eval()
        test_acc += models.accuracy(net,d_tst,d_test[1],verbose=False)
        #net.train()
        # Done with a task
    # Update main network
    print('Epoch  % 2d Loss: %2.5e Avg Acc: %2.5f'%(epoch,test_loss/task_count,test_acc/task_count))
    display.clear_output(wait=True)
    net.optimizer.zero_grad()
    test_loss.backward()
    net.optimizer.step()
    epoch+=1
    

Epoch   99 Loss: 1.00431e+00 Avg Acc: 0.95600


Now test the trained maml network and applying the adaption step to tasks sampled from the meta_test_ds dataset:

In [105]:
meta_test_kloader=KShotLoader(meta_test_ds,shots=shots,ways=ways)
test_acc = 0.0
task_count = 50
adapt_steps = 1
# Sample and train on a task
for task in range(task_count):
    d_train,d_test=meta_test_kloader.get_task()
    x_tr = d_train[0]
    y_tr_sh = torch.cat((torch.zeros(1,ways),torch.eye(ways)[d_train[1][1:]]))
    d_tr = torch.cat((x_tr,y_tr_sh),1).unsqueeze(0)
    net.adapt(d_tr)
    x_ts = d_test[0]
    y_ts_sh = torch.cat((torch.zeros(1,ways),torch.eye(ways)[d_test[1][1:]]))
    d_ts = torch.cat((x_ts,y_ts_sh),1).unsqueeze(0)
    d_tst = d_ts.transpose(0,1)
    test_preds = net(d_tst)
    test_acc += models.accuracy(net,d_tst,d_test[1],verbose=False)
    # Done with a task
net.train()
print('Avg Acc: %2.5f'%(test_acc/task_count))

Avg Acc: 0.94640
